In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import re
import joblib
import matplotlib.pyplot as plt
from pathlib import Path
from nltk.tokenize import word_tokenize
import nltk
import chardet
import grpc
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
from functools import lru_cache

C:\Users\Melek\PycharmProjects\MyDataset\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

from nltk.corpus import stopwords

# Türkçe stopwords'ler
turkish_stopwords = set(stopwords.words('turkish'))

# Genişletilmiş Türkçe stopwords listesi (ilk kodla aynı)
extended_turkish_stopwords = {
    'a', 'acaba', 'acep', 'adamakıllı', 'adeta', 'ait', 'altmış', 'altı',
    'ama', 'amma', 'anca', 'ancak', 'arada', 'artık', 'aslında', 'aynen', 'ayrıca',
    'az', 'açıkça', 'açıkçası', 'bana', 'bari', 'bazen', 'bazı', 'başkası',
    'belki', 'ben', 'benden', 'beni', 'benim', 'beri', 'beriki', 'beş',
    'bilcümle', 'bile', 'bin', 'binaen', 'binaenaleyh', 'bir', 'biraz',
    'birazdan', 'birbiri', 'birden', 'birdenbire', 'biri', 'birice', 'birileri',
    'birisi', 'birkaç', 'birkaçı', 'birkez', 'birlikte', 'birçok', 'birçoğu',
    'bir şey', 'bir şeyi', 'birşey', 'birşeyi', 'bitevi', 'biteviye',
    'bittabi', 'biz', 'bizatihi', 'bizce', 'bizcileyin', 'bizden', 'bize', 'bizi',
    'bizim', 'bizimki', 'bizzat', 'boşuna', 'bu', 'buna', 'bunda', 'bundan',
    'bunlar', 'bunları', 'bunların', 'bunu', 'bunun', 'buracıkta', 'burada',
    'buradan', 'burası', 'böyle', 'böylece', 'böylecene', 'böylelikle',
    'böylemesine', 'böylesine', 'büsbütün', 'bütün', 'cuk', 'cümlesi', 'da',
    'daha', 'dahi', 'dahil', 'dahilen', 'daima', 'dair', 'dayanarak', 'de', 'defa',
    'dek', 'demin', 'demincek', 'deminden', 'denli', 'derakap', 'derhal', 'derken',
    'değil', 'değin', 'diye', 'diğer', 'diğeri', 'doksan', 'dokuz',
    'dolayı', 'dolayısıyla', 'doğru', 'dört', 'edecek', 'eden', 'ederek', 'edilecek',
    'ediliyor', 'edilmesi', 'ediyor', 'elbet', 'elbette', 'elli', 'emme', 'en',
    'enikonu', 'epey', 'epeyce', 'epeyi', 'esasen', 'esnasında', 'etmesi', 'etraflı',
    'etraflíca', 'etti', 'ettiği', 'ettiğini', 'evleviyetle', 'evvel', 'evvela',
    'evvelce', 'evvelden', 'evvelemirde', 'evveli', 'eđer', 'eğer', 'fakat',
    'filanca', 'gah', 'gayet', 'gayetle', 'gayri', 'gayrı', 'gelgelelim', 'gene',
    'gerek', 'gerçi', 'geçende', 'geçenlerde', 'gibi', 'gibilerden', 'gibisinden',
    'gine', 'göre', 'gırla', 'hakeza', 'halbuki', 'halen', 'halihazırda', 'haliyle',
    'handiyse', 'hangi', 'hangisi', 'hani', 'hariç', 'hasebiyle', 'hasılı', 'hatta',
    'hele', 'hem', 'henüz', 'hep', 'hepsi', 'her', 'herhangi', 'herkes', 'herkesin',
    'hiç', 'hiçbir', 'hiçbiri', 'hoş', 'hulasaten', 'iken', 'iki', 'ila', 'ile',
    'ilen', 'ilgili', 'ilk', 'illa', 'illaki', 'imdi', 'indinde', 'inen', 'insermi',
    'ise', 'ister', 'itibaren', 'itibariyle', 'itibarıyla', 'iyi', 'iyice', 'iyicene',
    'için', 'iş', 'işte', 'kadar', 'kaffesi', 'kah', 'kala', 'kannımca',
    'karşın', 'katrilyon', 'kaynak', 'kaçı', 'kelli', 'kendi', 'kendilerine',
    'kendini', 'kendisi', 'kendisine', 'kendisini', 'kere', 'kez', 'keza',
    'kezalik', 'keşke', 'keţke', 'ki', 'kim', 'kimden', 'kime', 'kimi', 'kimisi',
    'kimse', 'kimsecik', 'kimsecikler', 'külliyen', 'kırk',
    'kısaca', 'lakin', 'leh', 'lütfen', 'maada', 'madem', 'mademki', 'mamafih',
    'mebni', 'meğer', 'meğerki', 'meğerse', 'milyar', 'milyon', 'mu',
    'mü', 'mi', 'mı', 'nasıl', 'nasılsa', 'nazaran', 'naşi', 'ne', 'neden',
    'nedeniyle', 'nedenle', 'nedense', 'nerde', 'nerden', 'nerdeyse', 'nere',
    'nerede', 'nereden', 'neredeyse', 'neresi', 'nereye', 'netekim', 'neye', 'neyi',
    'neyse', 'nice', 'nihayet', 'nihayetinde', 'nitekim', 'niye', 'niçin', 'o',
    'olan', 'olarak', 'oldu', 'olduklarını', 'oldukça', 'olduğu', 'olduğunu',
    'olmadı', 'olmadığı', 'olmak', 'olması', 'olmayan', 'olmaz', 'olsa', 'olsun',
    'olup', 'olur', 'olursa', 'oluyor', 'on', 'ona', 'onca', 'onculayın', 'onda',
    'ondan', 'onlar', 'onlardan', 'onları', 'onların', 'onu',
    'onun', 'oracık', 'oracıkta', 'orada', 'oradan', 'oranca', 'oranla', 'oraya',
    'otuz', 'oysa', 'oysaki', 'pek', 'pekala', 'peki', 'pekçe', 'peyderpey', 'rağmen',
    'sadece', 'sahi', 'sahiden', 'sana', 'sanki', 'sekiz', 'seksen', 'sen', 'senden',
    'seni', 'senin', 'siz', 'sizden', 'sizi', 'sizin', 'sonra', 'sonradan',
    'sonraları', 'sonunda', 'tabii', 'tam', 'tamam', 'tamamen', 'tamamıyla',
    'tarafından', 'tek', 'trilyon', 'tüm', 'var', 'vardı', 'vasıtasıyla', 've',
    'velev', 'velhasıl', 'velhasılıkelam', 'veya', 'veyahut', 'ya', 'yahut',
    'yakinen', 'yakında', 'yakından', 'yakınlarda', 'yalnız', 'yalnızca', 'yani',
    'yapacak', 'yapmak', 'yaptı', 'yaptıkları', 'yaptığı', 'yaptığını', 'yapılan',
    'yapılması', 'yapıyor', 'yedi', 'yeniden', 'yenilerde', 'yerine',
    'yetmiş', 'yine', 'yirmi', 'yok', 'yoksa', 'yoluyla', 'yüz', 'yüzünden',
    'zarfında', 'zaten', 'zati', 'zira', 'çabuk', 'çabukça', 'çeşitli', 'çok',
    'çokları', 'çoklarınca', 'çokluk', 'çoklukla', 'çokça', 'çoğu', 'çoğun',
    'çoğunca', 'çoğunlukla', 'çünkü', 'öbür', 'öbürkü', 'öbürü', 'önce', 'önceden',
    'önceleri', 'öncelikle', 'öteki', 'ötekisi', 'öyle', 'öylece', 'öylelikle',
    'öylemesine', 'öz', 'üzere', 'üç', 'şayet', 'şey', 'şeyden', 'şeyi', 'şeyler',
    'şu', 'şuna', 'şuncacık', 'şunda', 'şundan', 'şunlar', 'şunları', 'şunu',
    'şunun', 'şura', 'şuracık', 'şuracıkta', 'şurası', 'şöyle', 'şimdi', 'şöyle'
}

all_stopwords = turkish_stopwords.union(extended_turkish_stopwords)

In [3]:
# Zemberek GRPC bağlantısı
channel = grpc.insecure_channel('localhost:6789')
morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

# Lemma ve Kök bulma fonksiyonları
def get_lemmas(word):
    """Bir kelimenin lemmalarını (köklerini) Zemberek ile bulur"""
    try:
        response = morphology_stub.AnalyzeWord(z_morphology.WordAnalysisRequest(input=word))
        if response.analyses:
            return response.analyses[0].lemmas
        return []
    except:
        return []

def get_stem(word):
    """Bir kelimenin kökünü Zemberek ile bulur"""
    lemmas = get_lemmas(word)
    return lemmas[0] if lemmas else word
# Çoklu kelimeler için toplu kök bulma (performans iyileştirmesi)
def get_stems_batch(words):
    """Kelime listesi için kök bulma işlemini toplu yapar"""
    results = {}
    for word in words:
        if word not in results:
            results[word] = get_stem(word)
    return results


In [4]:
# Model yükleme
model_path = Path(r"C:\Users\Melek\yapayZeka\FilmTemaAnaliziProje\Models\tam_veri_seti_model4.pkl")
try:
    model = joblib.load(model_path)
    model_loaded = True
except:
    print(f"Model {model_path} konumundan yüklenemedi. Tahmin yapılmadan demo olarak çalışacaktır.")
    model_loaded = False


In [5]:
# Temalar (ilk kodla aynı sırada)
themes = [
    "romantik", "savaş", "bilim kurgu", "aksiyon", "dram", "fantastik", "gerilim", "suç",
    "tarih", "müzik", "komedi", "korku", "animasyon", "spor", "distoptik", "polisiye"
]

In [6]:
def clean_srt_file(file_path):
    """SRT dosyasını temizler (zaman damgalarını ve numaraları kaldırır)"""
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    current_line = ""

    for line in lines:
        line = line.strip()
        # Zaman damgası satırlarını ve numaralandırmayı atla
        if re.match(r'^\d+$', line) or '-->' in line:
            continue
        # Boş satırları atla
        if not line:
            if current_line:  # Eğer biriktirilen bir satır varsa ekle
                cleaned_lines.append(current_line)
                current_line = ""
            continue

        # Satırları birleştir (altyazılarda cümleler genelde birden fazla satıra bölünebilir)
        if current_line:
            current_line += " " + line
        else:
            current_line = line

    # Son satırı eklemeyi unutma
    if current_line:
        cleaned_lines.append(current_line)

    return cleaned_lines

In [7]:
from nltk import sent_tokenize


def split_into_sentences(lines):
    """Metni cümlelere böler"""
    sentences = []
    for line in lines:
        # NLTK'nın sent_tokenize fonksiyonunu kullanarak daha doğru cümle bölme
        line_sentences = sent_tokenize(line)
        for sentence in line_sentences:
            sentence = sentence.strip()
            if sentence:
                sentences.append(sentence)
    return sentences


In [8]:
def clean_sentence(sentence):
    """Cümleyi temizler"""
    # Altyazı notasyonlarını temizle ([gülüşmeler], (fısıldar) gibi)
    sentence = re.sub(r'\[.*?\]|\(.*?\)', '', sentence)

    # HTML etiketlerini kaldır
    sentence = re.sub(r'<.*?>', '', sentence)

    # Noktalama işaretlerini kaldır (sadece kelimeler, sayılar ve boşluklar kalsın)
    sentence = re.sub(r'[^\w\sğüşıöçĞÜŞİÖÇ0-9]', ' ', sentence)

    # Küçük harfe çevir (sayılar etkilenmez)
    sentence = sentence.lower()

    # Fazla boşlukları kaldır
    sentence = re.sub(r'\s+', ' ', sentence).strip()

    return sentence

In [27]:
def remove_stopwords_and_stem(sentence):
    words = word_tokenize(sentence)
    processed_words = []
    
    for word in words:
        if word not in all_stopwords and (len(word) > 2 or word.isdigit() or re.match(r'^\w+-\d+\.?\d*$', word)):
            if not word.isdigit() and not re.match(r'^\w+-\d+\.?\d*$', word):
                stemmed_word = get_stem(word)
                processed_words.append(stemmed_word)
            else:
                processed_words.append(word)
    
    return ' '.join(processed_words)

In [9]:
def remove_stopwords_and_stem(sentence, stem_cache=None):
    """Stopwords kaldırır ve kök bulma işlemi yapar"""
    if stem_cache is None:
        stem_cache = {}

    words = word_tokenize(sentence)
    processed_words = []

    # Cache'de olmayan kelimeler için kök bulma işlemi yap
    new_words = [w for w in words if w not in stem_cache and w not in all_stopwords
                 and (len(w) > 2 or w.isdigit() or re.match(r'^\w+-\d+\.?\d*$', w))]

    if new_words:
        new_stems = get_stems_batch(new_words)
        stem_cache.update(new_stems)

    # Cümledeki kelimeleri işle
    for word in words:
        if word not in all_stopwords and (len(word) > 2 or word.isdigit() or re.match(r'^\w+-\d+\.?\d*$', word)):
            if not word.isdigit() and not re.match(r'^\w+-\d+\.?\d*$', word):
                stemmed_word = stem_cache.get(word, get_stem(word))
                processed_words.append(stemmed_word)
            else:
                processed_words.append(word)

    return ' '.join(processed_words)

In [23]:
def create_chunks(sentences, chunk_size=25, stride=12):
    """Eğitimde kullanılan stratejiyle uyumlu overlap'li chunk'lar oluşturur"""
    chunks = []
    for i in range(0, len(sentences) - chunk_size + 1, stride):
        chunk = sentences[i:i + chunk_size]
        chunks.append(' '.join(chunk))

    # Son kalan cümleleri de ekle (eğitimdeki gibi tamamlanmamış chunk'ları da kullan)
    if len(sentences) % chunk_size != 0:
        remaining = sentences[-(len(sentences) % chunk_size):]
        chunks.append(' '.join(remaining))
    return chunks

In [25]:
def predict_theme_distribution(chunks):
    if not model_loaded:
        return {theme: np.random.uniform(0, 1) for theme in themes}

    predictions = model.predict(chunks)

    theme_counts = {}
    for theme in themes:
        theme_counts[theme] = 0

    for pred in predictions:
        theme_counts[pred] = theme_counts.get(pred, 0) + 1

    total = len(predictions)
    theme_percentages = {theme: (count / total) * 100 for theme, count in theme_counts.items()}

    return theme_percentages


In [26]:
def plot_theme_distribution(theme_percentages):
    sorted_themes = sorted(theme_percentages.items(), key=lambda x: x[1], reverse=True)
    labels = [item[0] for item in sorted_themes]
    values = [item[1] for item in sorted_themes]

    plt.figure(figsize=(12, 8))
    bars = plt.bar(labels, values, color='skyblue')
    plt.xlabel('Film Temaları')
    plt.ylabel('Yüzdelik (%)')
    plt.title('Film Tema Dağılımı')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height + 0.5,
                 f'{height:.1f}%', ha='center', va='bottom', rotation=0)

    return plt


In [27]:
from Frontend.film_analiz_arayuzu import process_subtitle_file


def read_file_with_auto_encoding(file_path):
    """Dosya karakter kodlamasını otomatik tespit ederek okur."""
    with open(file_path, 'rb') as f:
        raw_data = f.read()

    result = chardet.detect(raw_data)
    encoding = result['encoding']

    encodings_to_try = [
        encoding,
        'utf-8',
        'cp1254',
        'iso-8859-9',
        'latin-5',
        'iso-8859-1',
        'windows-1252'
    ]

    for enc in encodings_to_try:
        if enc is None:
            continue
        try:
            text = raw_data.decode(enc)
            print(f"{enc} kodlamasıyla başarıyla çözüldü.")
            return text
        except UnicodeDecodeError:
            continue

    raise ValueError("Dosya bilinen hiçbir karakter kodlamasıyla çözülemedi.")

def process_file(file):
    try:
        if hasattr(file, 'name'):
            file_content = read_file_with_auto_encoding(file.name)
        else:
            if isinstance(file, bytes):
                result = chardet.detect(file)
                encoding = result['encoding']
                try:
                    file_content = file.decode(encoding)
                except UnicodeDecodeError:
                    for enc in ['utf-8', 'cp1254', 'iso-8859-9', 'latin-5', 'iso-8859-1']:
                        try:
                            file_content = file.decode(enc)
                            break
                        except UnicodeDecodeError:
                            continue
                    else:
                        raise ValueError("Dosya hiçbir yaygın kodlamayla çözümlenemedi.")
            elif isinstance(file, str):
                file_content = file
            else:
                raise ValueError("Desteklenmeyen dosya formatı. Lütfen metin tabanlı bir SRT dosyası yükleyin.")

        processed_sentences = process_subtitle_file(file_content)

        if not processed_sentences:
            return "Dosyada geçerli bir metin bulunamadı. Lütfen dosya formatını kontrol edin.", None

        chunks = create_chunks(processed_sentences)
        theme_percentages = predict_theme_distribution(chunks)
        fig = plot_theme_distribution(theme_percentages)

        report = "🎬 Film Tema Analizi Sonuçları:\n\n"
        for theme, percentage in sorted(theme_percentages.items(), key=lambda x: x[1], reverse=True):
            report += f"• {theme.capitalize()}: %{percentage:.1f}\n"

        return report, fig
    except Exception as e:
        return f"Bir hata oluştu: {str(e)}", None

In [28]:
# Gradio arayüzü (orijinal kodla aynı)
with gr.Blocks(title="Film Tema Analizi") as app:
    gr.Markdown("# 🎬 Film Tema Analizi Aracı")

    with gr.Row():
        with gr.Column():
            file_input = gr.File(label="Altyazı Dosyası Yükle (.srt veya .txt)", file_types=[".srt", ".txt"])
            analyze_btn = gr.Button("Temaları Analiz Et", variant="primary")

        with gr.Column():
            result_text = gr.Textbox(label="Analiz Sonuçları", lines=12, interactive=False)

    chart_output = gr.Plot(label="Tema Dağılımı Grafiği")

    analyze_btn.click(
        fn=process_file,
        inputs=[file_input],
        outputs=[result_text, chart_output]
    )

In [29]:
# Uygulamayı başlat
if __name__ == "__main__":
    app.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
